
# POC 激活验证

In [1]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding, unpadding
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP


# x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_0.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)
del params

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
del model_detail

import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

# get a image
image = None
label = None
for i, (x_in, y_in) in enumerate(dataloader):
    image = x_in
    model(image)
    label = y_in
    break
del trainset, dataloader

In [2]:
# define the init process
def init_conv(data, data_size, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # print(weight.size())
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size[0], kernel_size[1])
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    # print("kernel_size, stride, padding:")
    # print(kernel_size, stride, padding)
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, 
                    stride=stride, padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    # print("conv input size:", data.size())
    x = operator(data.to("cuda"))

    return x


def initiate_node_feature(graph, image):
    ft = None 
    mask = graph.ndata['tag'] == 0
    out_channels = int(sum(mask))
    ft = torch.zeros((len(graph.nodes()), 1, 1, 28, 28))
    convd_size = torch.zeros((len(graph.nodes()), 2))
    for i in range(out_channels):
        kernel_size, stride, padding = graph.ndata['kernel_params'][i]
        # do conv
        res_ft = init_conv(image, None, graph.ndata['kernel_weight'][i], graph.ndata['bias'][i],
                kernel_size, stride, padding)
        # do relu
        relu_opt = torch.nn.functional.relu
        res_ft = relu_opt(res_ft)

        # do max pooling
        pooling = graph.ndata['pooling_params'][i]
        if pooling.all() != 0:
            # do max_pooling
            kernel_size, stride, pad, dilation, ceil_mode = pooling
            max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                padding=pad, dilation=dilation, ceil_mode=ceil_mode)
            
            res_ft = max_pooling_operator(res_ft)
        _, _, r, c = res_ft.size()
        ft[i] = res_ft
        convd_size[i] = torch.tensor([int(r), int(c)])
    graph.ndata['ft'] = ft.to("cuda")
    graph.ndata['ft_size'] = convd_size.to("cuda")



In [3]:
# init ft feature and get subg
with torch.no_grad():
    initiate_node_feature(g, image)
print(g.ndata['tag'])
# get node id
def nodes_with_feature_smaller_two(nodes):
    return nodes.data['tag'] <= 1
nodes_idx = g.filter_nodes(nodes_with_feature_smaller_two)
# print(nodes_idx)
subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
print(subg.nodes())
ft32 = None
# got the right subg of conv layers

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3],
       device='cuda:0')
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
       device='cuda:0')


In [9]:
def test_act_equals():
    # case 1, ones equal
    x = torch.ones((2,2,2))
    y = torch.ones((2,2,2))
    ret = equals(x, y)
    assert ret == True, "x != y in case 1"
    # case 2, ones not equal
    x = torch.ones((2,2,2))
    y = torch.rand((2,2,2))
    ret = equals(x, y)
    assert ret == False, "x == y in case 2"
    # case 3, zeros
    x = torch.zeros((2,2,2)).to("cuda")
    ret = equals(x, zeros=True)
    assert ret == True, "x != y in case 3"
test_act_equals()


In [5]:
def prepare_data(rdata, rsize):
    n = len(rdata)        
    cur_size = rsize[idx]
    width, height = cur_size
    ret_data = torch.zeros((n, 1, 1, width, height)).to("cuda")
    for idx in range(n):
        cur_data = rdata[idx]
        # convet (1, 1, 28, 28) to (1, 1, width, height)
        shaped_data = unpadding(cur_data, width, height)
        ret_data[idx] = shaped_data
    return ret_data


def prepare_conv_operator():
    pass

def do_operation():
    pass

def equals(x, y=None, zeros=False):
    if zeros:
        y = torch.zeros(x.size()).to("cuda")
    
    cmp_results = x == y
    if cmp_results.all() == True:
        return True
    else:
        return False


def my_reduce(nodes):
    # received data and their size
    mfeat = nodes.mailbox['m']
    msize = nodes.mailbox['ret_size']
    
    mf_size = mfeat.size()
    n_nodes = mf_size[0]
    n_channels = mf_size[1]

    # prepare ret_ft & ret_size, input size (1, 1, 28, 28)
    ret_ft = torch.zeros((n_nodes, 1, 1, 28, 28)).to("cuda")
    ret_size = torch.zeros((n_nodes, 2)).to("cuda")

    # for each node, do conv for received data
    for out_idx in range(n_nodes):

        # prepare received data and conv weight
        received_data = mfeat[out_idx]
        received_size = msize[out_idx]
        # skip if all zeros
        if equals(received_data, zeros=True):
            continue
        
        data = prepare_data(received_data, received_size)


        # prepare conv operator
        conv_opt = prepare_conv_operator()

        # do operation
        ft = do_operation()

        # save ft size
        cur_ret_size = ft.size()

        # update return data
        
    # return size is [n, 1, 1, 28, 28], reduced from [n, m, 1, 1, 28, 28]
    return {'h': ret_ft, 'reduce_ret_size': ret_size}

def my_message(edges):
    return {'m': edges.src['ft'], 'ret_size': edges.src['ft_size']}

def print_zeros_in_feat(key):
    print(key + ": ")
    zeros = []
    nonzero = []
    for id in subg.nodes():
        id = int(id)
        feat = subg.nodes[id].data[key]
        if torch.eq(feat, torch.zeros(feat.size()).to("cuda")).all() == True:
            zeros.append(id)
        else:
            nonzero.append(id)
    print("zeros - ", zeros)
    print("nonzeros - ", nonzero)


with torch.no_grad():
    subg.update_all(my_message, my_reduce)
    print("ret_size of data: ", subg.ndata['reduce_ret_size'])
    print("convd_size of data: ", subg.ndata['ft_size'])
    print("subg ft_size:", subg.ndata['ft_size'])
    print("subg ret_size:", subg.ndata['reduce_ret_size'])
    subg.ndata['ft_size'] += subg.ndata['reduce_ret_size']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    subg.ndata['ft'] += subg.ndata['h']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    cur_ft32 = subg.ndata['ft'][32]
    if ft32 is not None and torch.eq(cur_ft32, ft32).all():
        ft32 = cur_ft32
        print("ft32 equals!")
    else:
        ft32 = cur_ft32
        print("ft32 not equeal!")



AttributeError: 'NoneType' object has no attribute 'size'

In [ ]:
with torch.no_grad():
    subg.update_all(my_message, my_reduce)
    print("ret_size of data: ", subg.ndata['reduce_ret_size'])
    print("convd_size of data: ", subg.ndata['ft_size'])
    print("subg ft_size:", subg.ndata['ft_size'])
    print("subg ret_size:", subg.ndata['reduce_ret_size'])
    subg.ndata['ft_size'] += subg.ndata['reduce_ret_size']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    subg.ndata['ft'] += subg.ndata['h']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    cur_ft32 = subg.ndata['ft'][32]
    if ft32 is not None and torch.eq(cur_ft32, ft32).all():
        ft32 = cur_ft32
        print("ft32 equals!")
    else:
        ft32 = cur_ft32
        print("ft32 not equeal!")


In [10]:
cnn_params = None 
with open("./intermediate_data/params.json", "r") as f:
    cnn_params = json.load(f)
x_in = cnn_params['conv1']['in']
print("input is the same: ", torch.eq(torch.tensor(x_in), image).all() == True)

input is the same:  tensor(True)


In [11]:
ft = g.ndata['ft']
mask = g.ndata['tag'] == 0
print(ft[mask].size())
model_out = ft[mask]
out = torch.tensor(cnn_params['conv1']['out'])
print(out.size())
model_out = model_out.resize_((32, 28, 28)).to("cuda")
out = out.resize_((32, 28, 28)).to("cuda")
print(model_out.size(), out.size())
# conv1 out is the same 
from torch.nn.functional import relu
print("conv1 out is the same: ", torch.eq(model_out, relu(out)).all() == True)

torch.Size([32, 1, 1, 28, 28])
torch.Size([1, 32, 28, 28])
torch.Size([32, 28, 28]) torch.Size([32, 28, 28])
conv1 out is the same:  tensor(False, device='cuda:0')


In [12]:
# checking output by gnn
from torch.nn.functional import relu
conv2_out = torch.tensor(cnn_params['conv2']['relu'])
print(conv2_out.size())
gnn2_out = subg.ndata['ft'][32:64]
print(gnn2_out.size())
conv2_out = conv2_out.resize_((32, 28, 28)).to("cuda")
gnn2_out = gnn2_out.resize_((32, 28, 28)).to("cuda")
print(conv2_out.size(), gnn2_out.size())
import numpy as np
for id in range(len(conv2_out)):
    np.savetxt("./intermediate_data/cnn_out_%d.csv" % id, conv2_out[id].cpu().numpy(), delimiter=',')
    np.savetxt("./intermediate_data/gnn_out_%d.csv" % id, gnn2_out[id].cpu().numpy(), delimiter=',')


torch.Size([1, 32, 28, 28])
torch.Size([32, 1, 1, 28, 28])
torch.Size([32, 28, 28]) torch.Size([32, 28, 28])


In [13]:
equals = 0
from time import sleep
for out_idx in range(len(gnn2_out)):
    gf = gnn2_out[out_idx]
    for cf in conv2_out:
        eq_cond = torch.eq(gf, cf).all() == True
        zero_cond = (gf == 0).all() == True
        if zero_cond and eq_cond:
            print("data equals but are zero")
        elif eq_cond and not zero_cond:
            print("equals and not zeros")
            equals += 1
            
print("equals: ", equals)

data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
data equals but are zero
equals:  0


In [14]:
subg.ndata['ft'][32]

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 